# Eigenvalue problems (Unit 2.2) 

We solve the generalized eigenvalue problem
$$A u=\lambda M u$$,

where A comes from $\int \nabla u \nabla v$, and $M$ from $\int uv$, on the space $H_0^1$

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random 
from random import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))


We setup a stiffness matrix A and a mass matrix M, and declare a preconditioner for A:

In [2]:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$u_{n+1}=A^{−1}Mu_n,$$

where the Rayleigh quotient
$$\rho_n=\frac{\langle Au_n,u_n \rangle}{\langle Mu_n,u_n\rangle}$$

converges to the smallest eigen-value, with rate of convergence $\lambda_1/\lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{−1}$
by an approximate inverse $C^{−1}$

$$\rho_n=\frac{\langle Au_n,u_n\rangle}{\langle Mu_n,u_n\rangle}$$

$$w_n=C^{−1}(Au_n−\rho Mu_n)$$
$$u_{n+1}=u_n+\alpha w_n$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:
$$u_{n+1}=\arg \min_{v\in \lbrace u_n,w_n\rbrace} \frac{\langle Av,v\rangle}{\langle Mv,v\rangle}$$

This minimization problem can be solved by a small eigenvalue problem
$$ay=\lambda my$$

with matrices
$$a = \left(
\begin{array}[cc]
\ \langle Au_n,u_n\rangle & \langle Au_n,w_n\rangle\\
\langle Aw_n,u_n\rangle & \langle Aw_n,w_n\rangle
\end{array}
\right) \hspace{3em} m = \left(
\begin{array}[cc]
\ \langle Mu_n,u_n\rangle & \langle Mu_n,w_n\rangle\\
\langle Mw_n,u_n\rangle & \langle Mw_n,w_n\rangle
\end{array}
\right) $$

Then, the new iterate is

$$u_{n+1}=y_1 u_n+y_2 w_n$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.
### Implementation in NGSolve

Create some help vectors:


In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()


random initial condition, zero on the boundary

In [5]:
freedofs = fes.FreeDofs()    
for i in range(len(u.vec)):
    u.vec[i] = random() if freedofs[i] else 0   

In [6]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(asmall, b=msmall)
    # print (eval / pi**2)
    # print (evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    

23.829910971421537
2.053974392044293
2.0012098094341164
2.0001878572895864
2.000042827441213
2.000010785730308
2.0000027680511256
2.0000007202216157
2.000000188926783
2.0000000499070754
2.0000000132641667
2.0000000035616288
2.0000000009816525
2.0000000002936367
2.000000000109646
2.0000000000603455
2.0000000000471085
2.0000000000435487
2.000000000042591
2.000000000042334



### Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.


In [7]:
num = 5
u = GridFunction(fes, multidim=num)


Create list of help vectors:


In [8]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

freedofs = fes.FreeDofs()    
for v in u.vecs:
    for i in range(len(u.vec)):
        v[i] = random() if freedofs[i] else 0



Compute $num$ residuals, and solve small eigenvalue problem on $2num$-dimensional space

In [9]:
lams = num * [1]

asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)

for i in range(100):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        # r.data = 1/Norm(r) * r
        r *= 1/Norm(r)
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[0:num] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        r[:] = 0.0
        for k in range(2*num):
            r.data += float(evec[k,j]) * vecs[k]
        u.vecs[j].data = r
    
Draw (u)

0 : [11.28323989946033, 66.58817833362957, 72.42989310613859, 100.68201793850847, 112.02335630822931]
1 : [2.0428000043291425, 6.006937083366055, 11.422402704622396, 14.081151093495276, 20.147590230826278]
2 : [2.0013415433814954, 5.036901216488871, 5.317259953045415, 9.72657463387391, 10.988532890735376]
3 : [2.0001907536008483, 5.00256729386886, 5.013712761657964, 8.254465947518096, 10.133387014740721]
4 : [2.0000397767136384, 5.000234107850595, 5.001002591116617, 8.06286698450724, 10.04434031547282]
5 : [2.000009034915172, 5.000034953008672, 5.000136197259028, 8.023740836055715, 10.016806780686139]
6 : [2.0000020928353677, 5.000006477210253, 5.00002360006993, 8.01011216199555, 10.006490840043195]
7 : [2.0000004912530374, 5.000001292428562, 5.000004628590647, 8.004471865627943, 10.002525791875348]
8 : [2.0000001162054404, 5.000000269993664, 5.000000982494675, 8.00200457299012, 10.00098524456584]
9 : [2.000000027657475, 5.000000061075709, 5.000000222398725, 8.00090377013718, 10.000384

84 : [2.000000000042237, 5.0000000046722874, 5.000000005087632, 8.000000047998448, 10.000000156360535]
85 : [2.0000000000422378, 5.000000004672297, 5.0000000050876094, 8.000000047998418, 10.00000015636052]
86 : [2.0000000000422373, 5.0000000046723185, 5.000000005087648, 8.000000047998467, 10.000000156360548]
87 : [2.0000000000422355, 5.0000000046723665, 5.000000005087683, 8.000000047998402, 10.000000156360569]
88 : [2.0000000000422364, 5.000000004672179, 5.000000005087612, 8.000000047998395, 10.00000015636056]
89 : [2.000000000042237, 5.0000000046722555, 5.0000000050876725, 8.000000047998405, 10.000000156360546]
90 : [2.0000000000422364, 5.000000004672343, 5.000000005087675, 8.000000047998348, 10.000000156360505]
91 : [2.0000000000422373, 5.000000004672371, 5.00000000508773, 8.000000047998428, 10.000000156360542]
92 : [2.000000000042237, 5.000000004672312, 5.000000005087665, 8.0000000479984, 10.000000156360548]
93 : [2.000000000042238, 5.000000004672305, 5.000000005087656, 8.0000000479

The multidim-component select in the Visualization dialog box allows to switch between the components of the multidim-GridFunction.